In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
DATAPATH = "/home/kaariaa3/mscthesis/data/out.csv"

SYSTEM_PROMPT = """I want you to act as a programming teacher for an in-
troductory Dart course. Your students are programming
novices. I will provide some coding example exercises,
and it will be your job to critique them. Your responses 
should be written in simple English. Do not cite music 
lyrics or books. Do not include any greetings, be concise. 
Do not mention trigger words associated with mental or 
physical disorders, for example, weight loss or diet."""

CRITIQUE_TEMPLATE = """You are evaluating a programming exercise.

Intended theme: $THEME$
Intended topic: $TOPIC$
Intended programming concept: $CONCEPT$

Exercise description:
$TEXT$

Example solution:
$CODE$

Determine whether the exercise follows each requirement:

1. Theme
2. Topic
3. Programming concept

If it fails any requirement, explain WHY in detail.
Focus on concrete mismatches between instructions and content.
"""

# TEST!
TEMPLATE2 = """Analyze the exercise step by step.
Step 1 — Identify what the exercise is actually about.
Step 2 — Check alignment with the theme.
Step 3 — Check alignment with the topic.
Step 4 — Check alignment with the programming concept.
Step 5 — Provide a final explanation of failures.
"""

In [3]:
df = pd.read_csv("../../data/out.csv", sep=";")
df.head()

,title,topic,theme,concept,problemDescription,exampleSolution,The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),All metrics are correct (Yes/No)
0,Wildlife Spotting,wildlife spotting,outdoor activities,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes
1,Freeze Dance Game!,Freeze Dance,party games,user input,Write a program that asks the user if they wan...,"{'code': ""import 'dart:io';main() { print('Do...",yes,yes,yes,yes
2,Rye Bread Order,Rye bread,food,user input,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes
3,Tower of London!,Tower of London,historical landmarks,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,no,yes,no
4,Holiday Movies,watching holiday movies,Christmas,program output,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",no,yes,yes,no


In [4]:
label_col = df.columns[-1]
cond = df[label_col] == "no"

wrongs = df[cond]
wrongs.head()

,title,topic,theme,concept,problemDescription,exampleSolution,The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),All metrics are correct (Yes/No)
3,Tower of London!,Tower of London,historical landmarks,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,no,yes,no
4,Holiday Movies,watching holiday movies,Christmas,program output,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",no,yes,yes,no
11,Calculating the age of Looney Tunes characters.,Looney Tunes,cartoons,arithmetics,Write a program that asks the user for the cur...,"{'code': ""import 'dart:io'; main() { var cu...",yes,no,yes,no
13,Calculate the distance between two points in P...,Pyhä-Luosto National Park,nature destinations,variables,Write a program that asks the user for the coo...,"{'code': ""import 'dart:io'; main() { var x1...",yes,no,yes,no
15,Calculate the distance to Repovesi National Park.,Repovesi National Park,nature destinations,variables,Write a program that asks the user for their c...,"{'code': ""import 'dart:io'; main() { var cu...",yes,no,yes,no


In [8]:
explanations = {}

for idx, row in wrongs.iterrows():
    title, topic, theme, concept, problem_description, example_solution, *evals = row

    prompt = CRITIQUE_TEMPLATE

    # Explain all features
    prompt = (
        prompt
            .replace("$THEME$", theme)
            .replace("$TOPIC$", topic)
            .replace("$CONCEPT$", concept)
    )

    # Demand explanations from hallucinated topic/theme/concept
    """
    # Map from index to template label
    label = {0: "$THEME$", 1: "$TOPIC$", 2: "$CONCEPT$"}

    # Map template label to content
    rep = {"$THEME$": theme, "$TOPIC$": topic, "$CONCEPT$": concept}

    # Replace template labels with content

    # For finer explanations
    for i, evaluation in enumerate(evals):
        key = label.get(i, "")
        rep_str = rep.get(key, "")
        if key == "" or rep_str == "":
            continue

        if evaluation == "yes":
            prompt = prompt.replace(key, "")
            continue
        prompt = prompt.replace(key, f"{key[1:]}: " + rep_str + "\n")
    """

    prompt = (
        prompt
            .replace("$TEXT$", problem_description)
            .replace("$CODE$", example_solution)
    )

    explanations[idx] = prompt

    print(prompt)
    break


df = df.assign(Explanation=explanations).replace({np.nan: ""})

df.to_csv("out.csv", sep=";", index=False)


You are evaluating a programming exercise.

Intended theme: historical landmarks
Intended topic: Tower of London
Intended programming concept: user input

Exercise description:
Write a program that asks the user for their favorite historical landmark. If the user's favorite is the Tower of London, the program should print a message to the user ‘Tower of London is a great choice!’, where Tower of London is the landmark entered by the user. For example, with the input `Tower of London`, the program output is as follows:

```
What is your favorite historical landmark?
Tower of London
Tower of London is a great choice!
```

However, if the user enters a different landmark, the program should simply print the name of that landmark.

Example solution:
{'code': "import 'dart:io';main() {  print('What is your favorite historical landmark?');  var name = stdin.readLineSync();  if (name == 'Tower of London') {    print('$name is a great choice!');  } else {    print(name);  }}"}

Determine wheth

In [9]:
df.assign(Explanations=explanations).replace({np.nan: ""})

,title,topic,theme,concept,problemDescription,exampleSolution,The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),All metrics are correct (Yes/No),Explanations
0,Wildlife Spotting,wildlife spotting,outdoor activities,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,
1,Freeze Dance Game!,Freeze Dance,party games,user input,Write a program that asks the user if they wan...,"{'code': ""import 'dart:io';main() { print('Do...",yes,yes,yes,yes,
2,Rye Bread Order,Rye bread,food,user input,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,
3,Tower of London!,Tower of London,historical landmarks,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,
4,Holiday Movies,watching holiday movies,Christmas,program output,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,
5,Christmas Carol!,singing carols,Christmas,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';\nmain() {\n print...",yes,no,yes,no,Please generate an explanation why the followi...
6,Skiing Trip!,skiing,outdoor activities,user input,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,no,yes,no,Please generate an explanation why the followi...
7,Grand Canal Tour!,Grand Canal in Venice,historical landmarks,user input,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,
8,Treasure Hunt!,Treasure Hunt,party games,program output,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,no,yes,no,Please generate an explanation why the followi...
9,Canoeing Adventure!,canoeing,outdoor activities,program output,Write a program that asks the user for the nam...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,
